<a href="https://colab.research.google.com/github/Mreeb/YOLO-V8-CLASSIFICATION-ON_Dermnet/blob/master/1_Classification_YOLOv8_6_Classes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Connect Runtime To a GPU

In [1]:
!nvidia-smi

Wed Jan  3 14:12:31 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# Installing Libraies


In [2]:
!pip install --upgrade numpy
!pip install ultralytics==8.0.58 roboflow numpy==1.24.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 47.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.8/486.8 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 33.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Mounting Drive

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="C4pHQj9DsJmYjHwOLRr3")
project = rf.workspace("care-pftlv").project("skin-disease-classification-6o4dw")
dataset = project.version(3).download("folder")



loading Roboflow workspace...
loading Roboflow project...
Exporting format folder in progress : 85.0%
Version export complete for folder format



Extracting Dataset Version Zip to Skin-Disease-Classification-3 in folder:: 100%|██████████| 14458/14458 [00:04<00:00, 3422.50it/s]


In [1]:
from ultralytics import YOLO

model_path = 'yolov8x-cls.pt'  # @param {type:"string"}
data_path = '/content/Skin-Disease-Classification-3'  # @param {type:"string"}

epochs = 61  # @param {type:"integer"}
img_size = 640  # @param {type:"integer"}
amp = False # @param {type:"boolean"}
cache = True # @param {type:"boolean"}
resume = True # @param {type:"boolean"}

patience = 20 # @param {type:"integer"}
# Create YOLO model and train
model = YOLO(model_path)
model.train(data=data_path, epochs=epochs, imgsz=img_size, amp = amp, cache = cache, patience=patience)


ModuleNotFoundError: ignored

In [ ]:
path = "/content/runs/classify/train3/weights/best.pt"
from google.colab import files
files.download(path)